In [60]:
import re
from bert_predict_class import bert_predict, predict_class
import softmax_and_log_dicts

#### Prediction test!
This cell tests the prediction algorithm used

In [ ]:
sentence = 'The author of a keygen [MASK] a disassembler to look at the raw assembly code.'
pred = bert_predict(sentence)
scores = predict_class(pred, softmax_and_log_dicts.LOG_DICTS)
print(pred)
print(scores)

In [62]:
def get_short_class(long_class: str) -> str:
    if long_class == 'Component-Whole(e2,e1)' or long_class == 'Component-Whole(e1,e2)':
        return 'CW'
    if long_class == 'Instrument-Agency(e2,e1)' or long_class == 'Instrument-Agency(e1  ,e2)':
        return 'IA'
    if long_class == 'Member-Collection(e1,e2)' or long_class == 'Member-Collection(e2,e1)':
        return 'MC'
    if long_class == 'Cause-Effect(e2,e1)' or long_class == 'Cause-Effect(e1,e2)':
        return 'CE'
    if long_class == 'Entity-Destination(e2,e1)' or long_class == 'Entity-Destination(e1,e2)':
        return 'ED'
    if long_class == 'Content-Container(e2,e1)' or long_class == 'Content-Container(e1,e2)':
        return 'CC'
    if long_class == 'Message-Topic(e2,e1)' or long_class == 'Message-Topic(e1,e2)':
        return 'MT'
    if long_class == 'Product-Producer(e2,e1)' or long_class == 'Product-Producer(e1,e2)':
        return 'PP'
    if long_class == 'Entity-Origin(e2,e1)' or long_class == 'Entity-Origin(e1,e2)':
        return 'EO'

#### Class name decoding cell
Converts class names to two digit values

In [ ]:
with open('test_sentences.txt', 'r') as file:
    lines = file.readlines()

# set to start at sentence x of file, useful if script crashed midway
START_AT_SENTENCE_INDEX = 3299

for index, line in enumerate(lines):
    
    # zeilen mit maskiertem satz
    if index % 5 == 1:
        # we are not predicting here because we do not want to waste time on predicting sentences with class 'other'
        sentence = line
        sentence_number = line.split('\t')[0]

    # zeile mit label
    if index % 5 == 2:
        y = get_short_class(line.strip())

    # predict (bei kommentarzeile)
    if index % 5 == 3:
        # skip class: Other
        if y == None:
            continue
        # skip sentence if no mask token found
        if not re.search(r"\[MASK\]", sentence):
            continue
        # skip until wanted sentence
        if int(sentence_number) < START_AT_SENTENCE_INDEX:
            continue
        pred = bert_predict(sentence)
        scores = predict_class(pred, softmax_and_log_dicts.LOG_DICTS)

        with open('zero_shot_results_LOG.csv', 'a') as results:
            results.write(f"{sentence_number},{y},{scores[0][0]} \n")

Evaluate results:

In [12]:
with open('zero_shot_results_LOG_DICTS.csv', 'r') as csv:
    lines = csv.readlines()



    count = 0
    correct_count = 0
    for line in lines:

        # quick fix
        line = line.replace("CT", "MT")

        line = line.split(',')
        count += 1
        line[2] = line[2][:2]
        if line[1] == line[2]:
            correct_count += 1

print(f"Count: {count}")
print(f"Davon korrekt: {correct_count}")
print(correct_count / count)

Count: 4821
Davon korrekt: 1669
0.34619373573947315
